In [27]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough

In [3]:
llm = ChatOllama(model="llama3.1:latest",
                 temperature=0.7,
                 format="")

In [4]:
template="""Give me small report about {topic}"""

prompt_template=PromptTemplate.from_template(template)

llm_chain = prompt_template | llm

response = llm_chain.invoke({"topic": "Artificial Intelligence"})

In [5]:
response

AIMessage(content="Here's a brief report on Artificial Intelligence:\n\n**Overview**\n\nArtificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and problem-solving. AI has been around for decades, but recent advances in machine learning algorithms and data storage have made it increasingly powerful.\n\n**Key Features**\n\n* **Machine Learning**: AI systems use complex algorithms to learn from data and improve their performance over time.\n* **Natural Language Processing**: AI can understand and generate human language, enabling applications like chatbots and voice assistants.\n* **Computer Vision**: AI can interpret and understand visual information from images and videos.\n* **Robotics**: AI is used in robotics to control and navigate robots that interact with physical environments.\n\n**Applications**\n\n* **Virtual Assistants**: AI-powe

In [11]:
"Give me small report about {topic}".format(topic="Artificial Intelligence")  

'Give me small report about AI'

In [6]:
print(type(response))
print("\n==============\n")
print(response.response_metadata)
print("\n==============\n")
print(response.usage_metadata)
print("\n==============\n")
print(response.content)

<class 'langchain_core.messages.ai.AIMessage'>


{'model': 'llama3.1:latest', 'created_at': '2024-07-25T17:46:02.016745Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 6836633875, 'load_duration': 28566333, 'prompt_eval_count': 17, 'prompt_eval_duration': 219291000, 'eval_count': 386, 'eval_duration': 6587842000}


{'input_tokens': 17, 'output_tokens': 386, 'total_tokens': 403}


Here's a brief report on Artificial Intelligence:

**Overview**

Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and problem-solving. AI has been around for decades, but recent advances in machine learning algorithms and data storage have made it increasingly powerful.

**Key Features**

* **Machine Learning**: AI systems use complex algorithms to learn from data and improve their performance over 

In [7]:
previous_ai_message = response.content

messages = [HumanMessage("Give me small report about Artificial Intelligence"),
            AIMessage(previous_ai_message)]

system_template = """Answer the user prompt below using the chat history provided below."""

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{user_prompt}")
    ]
)

prompt_template.pretty_print()

================================ System Message ================================

Answer the user prompt below using the chat history provided below.

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{user_prompt}


In [8]:
prompt_value = prompt_template.invoke({"user_prompt": "what was your first sentence in your previous response.",
                                       "chat_history": messages})

print(type(prompt_value))
print("\n==============\n")
print(prompt_value.messages)

<class 'langchain_core.prompt_values.ChatPromptValue'>


[SystemMessage(content='Answer the user prompt below using the chat history provided below.'), HumanMessage(content='Give me small report about Artificial Intelligence'), AIMessage(content="Here's a brief report on Artificial Intelligence:\n\n**Overview**\n\nArtificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and problem-solving. AI has been around for decades, but recent advances in machine learning algorithms and data storage have made it increasingly powerful.\n\n**Key Features**\n\n* **Machine Learning**: AI systems use complex algorithms to learn from data and improve their performance over time.\n* **Natural Language Processing**: AI can understand and generate human language, enabling applications like chatbots and voice assistants.\n* **Computer Vision**: AI can interpret

In [9]:
llm_chain = prompt_template | llm

response = llm_chain.invoke({"user_prompt": "what was your first sentence in your previous response.",
                            "chat_history": messages})

print(response.content)

My first sentence in my previous response was:

"Here's a brief report on Artificial Intelligence:"


## RunnableLambda & RunnableParallel & RunnablePassThrough

In [25]:
def change_response(res):
    return "++START++\n" + res.content + "\n++END++"

change_response_func = RunnableLambda(change_response)


template="""Give me small report about {topic}"""

prompt_template=PromptTemplate.from_template(template)

llm_chain = prompt_template | llm | change_response_func

response = llm_chain.invoke({"topic": "Artificial Intelligence"})

print(response)

++START++
Here's a brief report on Artificial Intelligence:

**Introduction**

Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. AI has been rapidly evolving over the past few decades and has become an essential part of our daily lives.

**History**

The concept of AI dates back to ancient civilizations, but the modern era of AI began in the 1950s with the development of the first computer programs that could simulate human thought processes. The Dartmouth Summer Research Project on Artificial Intelligence in 1956 is considered a significant milestone in the history of AI.

**Key Aspects**

There are several key aspects of AI:

1. **Machine Learning**: This involves training algorithms to learn from data and improve their performance over time.
2. **Natural Language Processing (NLP)**: This enables computers to understand, interpret, and gen

In [31]:
def get_animal(a):
    return "lion"

def get_random_word(b):
    return "meat"

change_response_func = RunnableLambda(change_response)


template="""Use given words in one sentence: {animal} and {random_word}. Also consider the word provided by user: {user_prompt}"""

prompt_template=PromptTemplate.from_template(template)

retrieval = RunnableParallel(
    {
        "animal": RunnableLambda(get_animal), "random_word": RunnableLambda(get_random_word),
        "user_prompt": RunnablePassthrough()
    }
)

llm_chain = retrieval | prompt_template | llm

response = llm_chain.invoke("blood")

print(response.content)

Here's a sentence using all three words:

The lion, famished from its long hunt, devoured its prey with great relish, savoring the taste of fresh meat and the rich, crimson blood that still pulsed within its lifeless body.


## RAG Pipeline

# Summarization Pipeline

# Tool/Function Calling